In [2]:
import os
import abc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
import seaborn as sns

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings('ignore')

## Objectives

- Baselineを作る。
- 簡単な特徴量エンジニアリングを行う。
- バリデーション戦略について考える。
- 予測精度の評価方法を決める。

## Note

- Validation のラベルデータが配られたら、validation.csvと結合して、trainデータを増やす。

## Scores

## Load Data

In [25]:
# MEMO: 
# - コンペ終了1ヶ月前には sales_train_evaluation.csv が追加される。
# - train = sales_train_validation, test = sales_train_evaluation.
def read_data():
    files = ['calendar', 'sample_submission', 'sales_train_validation', 'sell_prices']
    
    if os.path.exists('/kaggle/input/m5-forecasting-accuracy'):
        data_dir_path = '/kaggle/input/m5-forecasting-accuracy'
        dst_data = {}
        for file in files:
            print(f'Reading {file} ....')
            dst_data[file] = pd.read_csv(data_dir_path + file + '.csv')
    else:
        data_dir_path = '../data/reduced/'
        dst_data = {}
        for file in files:
            print(f'Reading {file} ....')
            dst_data[file] = pd.read_pickle(data_dir_path + file + '.pkl')
    return dst_data.values()

# TODO: sales_train_evaluation.csv が公開されたらtestに代入する。
calendar, submission, train, sell_prices = read_data()
test = train.sample(10000).copy(deep=True)

Reading calendar ....
Reading sample_submission ....
Reading sales_train_validation ....
Reading sell_prices ....


In [21]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [22]:
# MEMO: 前処理と加工を行ったデータをキャッシュとして出力しておく。
def encode_categorical(df, cols):
    for col in cols:
        le = preprocessing.LabelEncoder()
        df[col] = le.fit_transform(df[col].values)
    return df


def add_time_features(df, dt_col):
    df[dt_col] = pd.to_datetime(df[dt_col])
    attrs = [
        "quarter",
        "month",
        "week",
        "day",
        "dayofweek",
        "is_year_end",
        "is_year_start",
        "is_quarter_end",
        "is_quarter_start",
        "is_month_end",
        "is_month_start",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df[dt_col].dt, attr).astype(dtype)

    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    return df


def encode_calendar(src_df):
    df = src_df.copy()
    drop_calendar_cols = ['weekday', 'year']
    df.drop(drop_calendar_cols, axis=1, inplace=True)
    # MEMO: N_Unique of event_name_1 == 31 and event_name_2 == 5.
    event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    df[event_cols] = df[event_cols].fillna('None')
    df = encode_categorical(df, event_cols)
    df[event_cols] = df[event_cols].astype('int8')
    return df



def melt_and_merge(train, test, calendar, sell_prices, use_cache=True):
    dir_path = 'processed'
    train_path = os.path.join(dir_path, 'train.pkl')
    test_path = os.path.join(dir_path, 'test.pkl')
    
    # If se cache
    if use_cache:
        if os.path.exists(train_path) and os.path.exists(test_path):
            train = pd.read_pickle(train_path)
            test = pd.read_pickle(test_path)
            return train, test
           
    # Encode Calender
    encoded_calender = encode_calendar(calendar)
    
    # Encode Train Data
    all_label_map = {}
    label_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] # Use for melt.
    # MEMO: ラベルは全データ共通なので、train/test/sell_prices のmappingに使える。
    for col in label_cols:
        all_labels = sorted(train[col].unique())
        all_label_map[col] = {label: i for i, label in enumerate(all_labels)}
    
    for col in label_cols:
        train[col] = train[col].map(all_label_map[col])
        test[col] = test[col].map(all_label_map[col])
        
        if col in ['store_id', 'item_id']:
            sell_prices[col] = sell_prices[col].map(all_label_map[col])
    
    train = pd.melt(train, id_vars=id_columns, var_name='d', value_name='sales')
    train = pd.merge(train, encoded_calender, how='left', on='d')
    train = pd.merge(train, sell_prices, how='left', on=['store_id', 'item_id', 'wm_yr_wk'])
    
    test = pd.melt(test, id_vars=id_columns, var_name='d', value_name='sales')
    test = pd.merge(test, encoded_calender, how='left', on='d')
    test = pd.merge(test, sell_prices, how='left', on=['store_id', 'item_id', 'wm_yr_wk'])
    # MEMO: sell_price を直近価格で過去の値を埋める。
    train['sell_price'] = train.groupby('item_id')['sell_price'].bfill()
    test['sell_price'] = test.groupby('item_id')['sell_price'].bfill()
    
    # Save cache
    train = train.pipe(reduce_mem_usage)
    test = test.pipe(reduce_mem_usage)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)

    train.to_pickle(train_path)
    test.to_pickle(test_path)
    return train, test

train, test = melt_and_merge(train, test, calendar, sell_prices, use_cache=True)

In [23]:
print(train.shape)
display(train.head())

print(test.shape)
display(test.head())

(58327370, 19)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,d_1,0,11101,1,1,19,2,3,1,0,0,0,9.578125
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,d_1,0,11101,1,1,19,2,3,1,0,0,0,3.970703
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,d_1,0,11101,1,1,19,2,3,1,0,0,0,2.970703
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,d_1,0,11101,1,1,19,2,3,1,0,0,0,4.339844
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,d_1,0,11101,1,1,19,2,3,1,0,0,0,2.980469


(19130000, 19)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_3_203_WI_2_validation,815,2,0,8,2,d_1,0,11101,1,1,19,2,3,1,0,0,0,4.500000
1,FOODS_3_642_WI_2_validation,1254,2,0,8,2,d_1,0,11101,1,1,19,2,3,1,0,0,0,3.480469
2,HOUSEHOLD_2_035_CA_3_validation,2568,6,2,2,0,d_1,4,11101,1,1,19,2,3,1,0,0,0,6.468750
3,FOODS_3_692_CA_2_validation,1304,2,0,1,0,d_1,3,11101,1,1,19,2,3,1,0,0,0,0.879883
4,FOODS_3_529_WI_1_validation,1141,2,0,7,2,d_1,0,11101,1,1,19,2,3,1,0,0,0,3.980469


## Feature Engineering

- shiftするだけの特徴量
- 予測日数shiftした後、rollingした統計情報
- calendarの情報
    - 翌日休日フラグ
        - CalendarShiftFeature
    - 連休日数
        - 翌日休日フラグを前後にshiftすればよさそう。
- 当該月の特徴量
    - 1日の売上
    - 7, 10, 15, 20日時点の累積売上
        - Calendarにデータを作って、groupby したほうがいいだろうなー。
- 過去１ヶ月での、スーパー全体の売上のうちの自分の売上の割合
    - 相対的にみてよく売れる商品ですよ。というのを言える。
        - てことは、個数で見たほうがいいかもしれない。

In [6]:
class Feature(metaclass=abc.ABCMeta):
    save_dir = "features"
    
    def __init__(self):
        self.name = self.__class__.__name__
        self.train_path = os.path.join(self.save_dir, f"{self.name}_train.pkl")
        self.test_path = os.path.join(self.save_dir, f"{self.name}_test.pkl")

    def run(self, train, test, use_cache=True):
        if use_cache:
            if os.path.exists(self.train_path) and os.path.exists(self.test_path):
                return pd.read_pickle(self.train_path), pd.read_pickle(self.test_path)
        
        self.train = self.create_features(train)
        self.test = self.create_features(test)
        
        if not os.path.exists(self.save_dir):
            os.mkdir(self.save_dir)
            
        self._save()
        
        return self.train, self.test

    @abc.abstractmethod
    def create_features(self, df):
        raise NotImplementedError

    def _save(self):
        self.train.to_pickle(self.train_path)
        self.test.to_pickle(self.test_path)

In [10]:
class ShiftedFeautre(Feature):
    def create_features(self, df):
        PRED_INTERVAL = 28
        shift_cols = ['sales', 'sell_price']
        shift_size = [0, 1, 2, 3, 5, 7, 14, 28, 30]

        grouped = df.groupby('id')[shift_cols]
        for size in tqdm(shift_size):
            size = PRED_INTERVAL + size
            col_name = [f'shifted_{c}_t{size}' for c in shift_cols]
            df[col_name] = grouped.shift(size)
        return df

In [11]:
%%time
train, test = ShiftedFeautre().run(train, test, use_cache=True)


CPU times: user 1min 12s, sys: 1min 11s, total: 2min 24s
Wall time: 2min 56s


In [14]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,...,shifted_sales_t33,shifted_sell_price_t33,shifted_sales_t35,shifted_sell_price_t35,shifted_sales_t42,shifted_sell_price_t42,shifted_sales_t56,shifted_sell_price_t56,shifted_sales_t58,shifted_sell_price_t58
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,d_1,0,11101,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,d_1,0,11101,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,d_1,0,11101,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,d_1,0,11101,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,d_1,0,11101,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# MEMO: Calculate Slope Function.
from scipy.stats import linregress

slope_func= lambda x: linregress(np.arange(len(x)), x)[0]

class RolledFeautre(Feature):
    def create_features(self, df):
        PRED_INTERVAL = 28
        rolling_cols = ['sales', 'sell_price']
        rolling_size = [3, 7, 14, 30, 60, 180]
        agg_funcs = ['mean', 'std', 'sum', 'min', 'max', 'skew', 'kurt']

        grouped = df.groupby('id')[rolling_cols]
        
        for size in tqdm(rolling_size):
            rolled = grouped.shift(PRED_INTERVAL).rolling(size).agg(agg_funcs)
            col_name = [f'{c[0]}_{c[1].upper()}_rolling_t{size}' for c in rolled.columns]
            rolled.columns = col_name
            df[col_name] = rolled
        return df

NameError: name 'Feature' is not defined

In [ ]:
%%time
train, test = RolledFeautre().run(train, test, use_cache=True)

## Training Model and Prediction

### Vlidation

- 一番最後の28日を検証用としてholdoutする。
- 残りのデータでCVを生成する。

In [ ]:
class CustomTimeSeriesSplitter:
    def __init__(self, n_splits=5, train_days=80, test_days=20, dt_col="date"):
        self.n_splits = n_splits
        self.train_days = train_days
        self.test_days = test_days
        self.dt_col = dt_col

    def split(self, X, y=None, groups=None):
        sec = (X[self.dt_col] - X[self.dt_col][0]).dt.total_seconds()
        duration = sec.max() - sec.min()

        train_sec = 3600 * 24 * self.train_days
        test_sec = 3600 * 24 * self.test_days
        total_sec = test_sec + train_sec
        step = (duration - total_sec) / (self.n_splits - 1)

        for idx in range(self.n_splits):
            train_start = idx * step
            train_end = train_start + train_sec
            test_end = train_end + test_sec

            if idx == self.n_splits - 1:
                test_mask = sec >= train_end
            else:
                test_mask = (sec >= train_end) & (sec < test_end)

            train_mask = (sec >= train_start) & (sec < train_end)
            test_mask = (sec >= train_end) & (sec < test_end)

            yield sec[train_mask].index.values, sec[test_mask].index.values

    def get_n_splits(self):
        return self.n_splits
    
def plot_cv_indices(cv, X, y, dt_col, lw=10):
    n_splits = cv.get_n_splits()
    _, ax = plt.subplots(figsize=(20, n_splits))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            X[dt_col],
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=plt.cm.coolwarm,
            vmin=-0.2,
            vmax=1.2,
        )

    # Formatting
    MIDDLE = 15
    LARGE = 20
    ax.set_xlabel("Datetime", fontsize=LARGE)
    ax.set_xlim([X[dt_col].min(), X[dt_col].max()])
    ax.set_ylabel("CV iteration", fontsize=LARGE)
    ax.set_yticks(np.arange(n_splits) + 0.5)
    ax.set_yticklabels(list(range(n_splits)))
    ax.invert_yaxis()
    ax.tick_params(axis="both", which="major", labelsize=MIDDLE)
    ax.set_title("{}".format(type(cv).__name__), fontsize=LARGE)
    return ax

In [ ]:
cv_params = {
    "n_splits": 5,
    "train_days": 365 * 2,
    "test_days": DAYS_PRED,
    "dt_col": dt_col,
}
cv = CustomTimeSeriesSplitter(**cv_params)
# Plotting all the points takes long time.
plot_cv_indices(cv, data.iloc[::1000][[dt_col]].reset_index(drop=True), None, dt_col)

## Submission